# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
The problem at hand is a classification problem as the output space is boolean (pass/fail) and therefore discrete.
A regression Problem would be defined by a continuous output space like f.ex. giving an estimate for the average of all grades of a student in real numbers. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
from __future__ import division
# TODO: Calculate number of students
n_students = len(student_data.axes[0])

# TODO: Calculate number of features
# subtract by one because the labels are still included
n_features = len(student_data.axes[1])-1


# TODO: Calculate passing students
n_passed = len( student_data[student_data['passed']=='yes'].axes[0])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'].axes[0])

# TODO: Calculate graduation rate
grad_rate = 100*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
# Plot the training points
import matplotlib.pyplot as plt
%matplotlib inline


#I found out the failures column must have a strong relationship with the labels when I implemented a Decision Tree Classifier
#So I wanted to do a bit of analysis here
failure_yes=[]
failure_no=[]
for index, v in enumerate(y_all):
    if v == "no":
        failure_no.append(X_all["failures"][index])
    else:
        failure_yes.append(X_all["failures"][index])

#this could be done more efficiently with a single loop, but i wanted to try python's lambdas
#maybe there is even some function to create a histogram?
        #is there currying for python?
count3s  = lambda x : 1 if x==3 else 0
count2s  = lambda x : 1 if x==2 else 0
count1s  = lambda x : 1 if x==1 else 0
count0s  = lambda x : 1 if x==0 else 0

print np.sum(map (count3s,failure_no))/len(failure_no)
print np.sum(map (count2s,failure_no))/len(failure_no)
print np.sum(map (count1s,failure_no))/len(failure_no)
print np.sum(map (count0s,failure_no))/len(failure_no)

print np.sum(map (count3s,failure_yes))/len(failure_yes)
print np.sum(map (count2s,failure_yes))/len(failure_yes)
print np.sum(map (count1s,failure_yes))/len(failure_yes)
print np.sum(map (count0s,failure_yes))/len(failure_yes)

from IPython.display import display # Allows the use of display() for DataFrames
display(X_all["failures"].describe())


Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
0.0923076923077
0.107692307692
0.2
0.6
0.0150943396226
0.011320754717
0.0905660377358
0.883018867925


count    395.000000
mean       0.334177
std        0.743651
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.000000
Name: failures, dtype: float64

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
import numpy as np
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train


ratio= 1-(num_train/X_all.shape[0])


# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, test_size=ratio, random_state=2)



# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **


#### 1. KNN
is generally applicable to categorization problems. It can be adapted to most problems using an appropriate distance function (besides other parameters which I think are not equally important). It should be interesting how well the lazy learning algorithm performs and how quickly the model can predict values. I expect learning times to be significantly better than the ones of the other two. The downside is, of course, that prediction times may be a lot slower. Considering the data, I can imagine that an appropriate distance function should perform quite good. In this domain I'd expect good students to be in clusters, considering their features.  F.ex. usually parents with high incomes have successful children as well.Successful industry applications include:


* Nearest Neighbor based Content Retrieval. 
  KNN has been successfully used to retrieve images by similarity from a large database.  
* OCR
* Protein-Protein Interaction and structure prediction (together with SVMs)
* breast cancer prediction
* medical data mining

sources:
* https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/
* http://vlm1.uta.edu/~athitsos/nearest_neighbors/
* http://arxiv.org/abs/1307.4717
* http://www.ncbi.nlm.nih.gov/pmc/articles/PMC2243774/
* https://www.researchgate.net/publication/270163293_Application_of_k-Nearest_Neighbour_Classification_in_Medical_Data_Mining


#### 2. Decision trees
can be used for categorization as well. They provide a quick way to create a model because they automatically choose the most important features first due to their learning algorithm. This makes feature selection easier to do as one could try to omit the features the tree puts in the end. Also decision trees work well on data that's not linearly separable. As an additional reason, the prediction time should be relatively low if the model isn't overly complex. Which leads to one of the disadvantages as decision trees tend to overfit easily when they grow too much and split more than necessary. To mitigate this effect, post-pruning of the tree needs to be applied. The last advantage of a decision tree is, that it induces a set of decision rules that can be used to explain the reasoning behind a decision, in this particular context I think this could be very beneficial. 
Successful applications of decision trees include the classification of galaxies and stars, filtering noise from hubble space telescope images, for power system security assessment and power stability prediction. Furthermore Decision Trees have applications in pharmacology for classification of drugs and applications in financial analysis. 

sources:
* http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html
* http://www.simafore.com/blog/bid/62333/4-key-advantages-of-using-decision-trees-for-predictive-analytics
* http://120.107.180.177/1832/9901/099-1-03p.pdf (not used for my discussion above, I just found it noteworthy that someone actually tried to use DTCs for student intervention)

#### 3. SVMs
are another obvious choice for classification. They are generally good in complicated domains with clear separation of data but tend to perform worse on when classes overlap. Also, the larger the training data gets and the more features are added, the slower SVMs become in learning and prediction times. SVMs are prone to overfitting when data is noisy. The most challenging task is to find a suitable Kernel function for the given data as this requires domain knowledge and careful parameter tuning. For the given data, I think using an SVC should be interesting because with the right tuning and kernel function it could outperform the other two classifiers in terms of accuracy. On the negative side I expect the training and prediction performance to be worse than that of the other two.  SVMs have been successfully applied for OCR,text categorization,image classification, and in biological sciences f.ex. to classify proteins. Another field of application I find particularly interesting is in Intrusion Detection Systems. 

* https://en.wikipedia.org/wiki/Support_vector_machine#Applications
* http://www.clopinet.com/SVM.applications.html
* http://www-labs.iro.umontreal.ca/~pift6080/H09/documents/papers/svm_tutorial.ppt
* http://www.ijais.org/proceedings/icwac/number3/495-1342



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
# TODO: Initialize the three models
clf_A = KNeighborsClassifier(n_neighbors=3)
clf_B = DecisionTreeClassifier(random_state=999999977)
clf_C = SVC(random_state=3235436)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)

train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)

train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0080 seconds.
F1 score for training set: 0.8504.
Made predictions in 0.0050 seconds.
F1 score for test set: 0.7656.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6557.
Training a SVC using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8905.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8392.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0030 seconds
Made predictions in 0.0120 seconds.
F1 score for training set: 0.8669.
Made predictions in 0.0110 seconds.
F1 score for test set: 0.7826.
Training a DecisionTreeClassifier using a training set 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - KNN(k=3)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010s                 |  0.0010s               | 0.8321           | 0.7482          |
| 200               | 0.0010s                 |  0.0030s               | 0.8966           | 0.7413          |
| 300               | 0.0010s                 |  0.0050s               | 0.8807           | 0.7313          |

** Classifer 2 - DecisionTreeClassifier

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010s                 | 0s                     |  1               | 0.6555          |
| 200               | 0.0020s                 | 0s                     |  1               | 0.75            |
| 300               | 0.0010s                 | 0s                     |  1               | 0.6167          |

** Classifer 3 - SVC 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010s                 | 0.0010s                | 0.8777           | 0.7746          |
| 200               | 0.0030s                 | 0.0020s                | 0.8679           | 0.7815          |
| 300               | 0.0060s                 | 0.0040s                | 0.8761           | 0.7838          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
(the results in the table are not exactly the ones calculated above, because I changed the random_state variable of train_test_split a few times)

My first intuition was to create a DecisionTreeClassifier because it is significantly faster than the other classifiers and it identifies features of high importance automatically. So I hoped I would gain more insight into the structure of the data.
Indeed I think I found some interesting facts about the data.  
The best tree found by training on the training set always splits on only one feature and ignores all the others. That particular feature is the "failures" feature. Using only the one feature the tree is able to achieve a comparably high F1 Score of about 8.5 which is as good as SVC or KNN can achieve. 
An interesting fact I noticed was that the F1 Score for the DecisionTreeClassifier is extremely sensitive to the random_seed I set in the train_test_split method. Initially I was unsure as to why this is the case but now it seems clear to me. Since the tree only uses one feature, it obviously very much depends on how equally the "good data" in respect to the failures feature is split up in testing and training set. As a conclusion I'd say that using a decision tree is bad idea in this context as it captures very little of the information because it instantly overfits the data and the F1 Score becomes pretty much useless. 

As KNN has the worst prediction times because of its lazy training nature and its F1 score on the testing set is worse than that of the SVC, I will try and use the SVC as a classifier.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

The set of students can be thought of consisting of two different classes. One class contains all students who were successful and passed, and the other class contains all students who failed. The SVC (StateVectorClassifier) needs to make a decision for every new student to which class she belongs. To understand how the SVC works one can visualize the students as points on a 2D plane. One could then see one cluster or group of all students who failed and one distinct group or point cloud containing all students who passed. The SVC will then try to draw a line between the two classes which clearly separates them from another. 
The line is chosen so that the distance from it to the nearest data point (student) on each side is maximized. In other words it tries to find the maximum margin between the two classes. 

Of course not all data can be separated by a simple line in that way. The solution for that is to use hyper planes and different kernel functions, which are complicated mathematical terms that need not to be explained here. Suffice to say that these methods are clever tricks to adapt the SVC so that the basic principle explained above stays the same.

sources:
* https://en.wikipedia.org/wiki/Support_vector_machine

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
#I let my Decision Tree Code Block here as I have learned some interesting facts from it
#Ihope thats ok, the SVC implementation will follow in the next code block.
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import f1_score,make_scorer
# TODO: Create the parameters list you wish to tune

def runTraining(X_train, y_train, X_test, y_test, parameters, clf):
    
    # TODO: Make an f1 scoring function using 'make_scorer' 
    f1_scorer = make_scorer(f1_score, pos_label='yes')

    # TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
    grid_obj = grid_search.GridSearchCV(clf, parameters,scoring=f1_scorer, cv=2,n_jobs=4)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    grid_obj = grid_obj.fit(X_train, y_train)

    # Get the estimator
    clf = grid_obj.best_estimator_
    
    # Report the final F1 score for training and testing after parameter tuning
    print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
    return clf
    

print "Run training:"
parameters = {'max_depth':[1,2,3,4]
                 ,'min_samples_split':[1,2,5],'max_leaf_nodes':[2,3,4],'splitter':['best','random']
                 }
# TODO: Initialize the classifier
clf = DecisionTreeClassifier(
       criterion='entropy', max_depth=None, min_samples_split=2,
       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None,
       random_state=6578, max_leaf_nodes=None, class_weight=None, presort=False)
    
final_tree= runTraining(X_train,y_train, X_test, y_test,parameters, clf)

#print some interesting information about the tree with code taken from the sklearn documentation
from sklearn import tree

estimator=final_tree

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()


Run training:
Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8092.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8169.
The binary tree structure has 3 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 32] <= 0.5s else to node 2.
	node=1 leaf node.
	node=2 leaf node.
()


C:\Users\ganjalf\Anaconda2\lib\site-packages\ipykernel\__main__.py:80: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\ganjalf\Anaconda2\lib\site-packages\ipykernel\__main__.py:76: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [13]:
#now train and implement the SVC
print "Total Feature Count:"+str( X_train.shape[1])

print "Run training and evaluation on all features:"
parameters = {'C':[.0001,.001,.01,.1,1]
              ,'gamma':[.00001,.0001,.001,.01]
              ,'kernel':['rbf']
              }
# TODO: Initialize the classifier
clf = SVC(random_state=6578)

clf= runTraining(X_train,y_train, X_test, y_test,parameters, clf)
print "Support Vectors:"
print clf.support_vectors_

print "--------------------"



X_train_redu_ft_set, X_test_redu_ft_set, y_train_redu_ft_set, y_test_redu_ft_set = train_test_split(X_all,y_all, test_size=ratio, random_state=94687681)

#now that I know, that the "failures" feature has to be important, Ill try selecting the KBest explicitly without it, just to get a feeling for how good 
#the SVC can get. I am interested if I still can get good results.
del X_train_redu_ft_set['failures']
del X_test_redu_ft_set['failures']

ch2 = SelectKBest(chi2, k=10)

X_train_redu_ft_set=ch2.fit_transform(X_train_redu_ft_set, y_train_redu_ft_set)
X_test_redu_ft_set= ch2.transform(X_test_redu_ft_set)

#now run the training
print "Feature subset(failures removed):"+str( X_train_redu_ft_set.shape[1])
print "--------------------"
print "Run training with reduced feature set- without the failures column:"

parameters = {'C':[.0001,.001,.01,.1,1]
              ,'gamma':[.00001,.0001,.001,.01]
              ,'kernel':['rbf','poly'],'degree':[1,2,3,4]
              }
# TODO: Initialize the classifier
clf = SVC(random_state=6578)
clf=runTraining(X_train_redu_ft_set,y_train_redu_ft_set,X_test_redu_ft_set,y_test_redu_ft_set, parameters, clf)
print "Support Vectors:"
print clf.support_vectors_


#and one last time with all features and a kbest filter 
X_train_redu_ft_set, X_test_redu_ft_set, y_train_redu_ft_set, y_test_redu_ft_set = train_test_split(X_all,y_all, test_size=ratio, random_state=94687681)


ch2 = SelectKBest(chi2, k=35)

X_train_redu_ft_set=ch2.fit_transform(X_train_redu_ft_set, y_train_redu_ft_set)
X_test_redu_ft_set= ch2.transform(X_test_redu_ft_set)

#now run the training
print "Feature subset:"+str( X_train_redu_ft_set.shape[1])
print "--------------------"
print "Run training with reduced feature set:"

parameters = {'C':[.0001,.001,.01,.1,1]
              ,'gamma':[.00001,.0001,.001,.01]
              ,'kernel':['rbf','poly'],'degree':[1,2,3,4]
              }
# TODO: Initialize the classifier
clf = SVC(random_state=6578)
clf=runTraining(X_train_redu_ft_set,y_train_redu_ft_set,X_test_redu_ft_set,y_test_redu_ft_set, parameters, clf)
print "Support Vectors:"
print clf.support_vectors_



Total Feature Count:48
Run training and evaluation on all features:
Made predictions in 0.0230 seconds.
Tuned model has a training F1 score of 0.8337.
Made predictions in 0.0090 seconds.
Tuned model has a testing F1 score of 0.8289.
Support Vectors:
[[ 1.  0.  1. ...,  2.  3.  0.]
 [ 1.  0.  1. ...,  2.  4.  0.]
 [ 1.  0.  0. ...,  2.  4.  0.]
 ..., 
 [ 1.  0.  1. ...,  1.  1.  2.]
 [ 0.  1.  1. ...,  4.  5.  0.]
 [ 1.  0.  0. ...,  2.  1.  5.]]
--------------------
Feature subset(failures removed):10
--------------------
Run training with reduced feature set- without the failures column:
Made predictions in 0.0040 seconds.
Tuned model has a training F1 score of 0.8000.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8302.
Support Vectors:
[[  0.   1.   0. ...,   0.   2.   0.]
 [  0.   1.   0. ...,   0.   4.   4.]
 [  0.   1.   0. ...,   1.   2.  14.]
 ..., 
 [  1.   0.   1. ...,   0.   4.   0.]
 [  0.   1.   0. ...,   0.   5.  10.]
 [  1.   0.   0. ...,   0

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
For my final model I got three different results because I wanted to try out feature selection and analyse the importance of the failures feature. The first SVC, I applied to the full feature set. It seems to perform best using an RBF kernel with a final score of .8289. I am a bit skeptical about the high training score, maybe the model is overfitting the data. 
The reduced feature set SVC perform better using a poly kernel. 
Out of curiosity I created one SVC without the failures feature (see discussion above) to see if the other features would suffice for good results and indeed they did. The second SVC, that explicitly excluded the failures feature scored .8302.
Most interestingly, the last SVC performed best with a reduced feature set of 35 features it scored .8408. 
Which I think is quite good. All of the scores exceed the untrained model as expected. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.